**Claim extraction from text documents**

By fine tuning transformers

In [1]:
# load the dataset to huggingface class
from datasets import load_dataset

"""the data consists of 395057 entries with
 - 209612 being no claims
 - 185445 being claims
 """

# 70 / 30 split
train = load_dataset('csv', data_files='data/combined_dataset.csv', split='train[:70%]').rename_column("y", "labels")
test = load_dataset('csv', data_files='data/combined_dataset.csv', split='train[70%:]').rename_column("y", "labels")

print(train, test)

/home/cas/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-97b16aeddd44bfca
Found cached dataset csv (/home/cas/.cache/huggingface/datasets/csv/default-97b16aeddd44bfca/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-97b16aeddd44bfca
Found cached dataset csv (/home/cas/.cache/huggingface/datasets/csv/default-97b16aeddd44bfca/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['Unnamed: 0', 'claim', 'labels'],
    num_rows: 276540
}) Dataset({
    features: ['Unnamed: 0', 'claim', 'labels'],
    num_rows: 118517
})


In [2]:
# get tokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(batch):
    return tokenizer(
        batch["claim"], 
        padding="max_length", 
        truncation=True, 
        return_tensors='pt',
        return_attention_mask=True
    )

In [3]:
# tokenize all train claims in advance
tokenized_train = train.map(tokenize_function, remove_columns=["Unnamed: 0", "claim"]) # batched=True,

100%|██████████| 276540/276540 [01:22<00:00, 3347.46ex/s]


In [4]:
tokenized_test = test.map(tokenize_function, remove_columns=["Unnamed: 0", "claim"]) # batched=True

100%|██████████| 118517/118517 [00:35<00:00, 3357.77ex/s]


In [5]:
tokenized_train

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 276540
})

In [17]:
# this is what I think is called during training
# it has the size of 3, so whats the problem?? -> maybe with batches of tokenizer?
a = tokenized_test.__getitem__(0)
a.keys()

dict_keys(['labels', 'input_ids', 'attention_mask'])

In [7]:
# load the distilbert to finetune from huggingface
config = BertConfig.from_pretrained( 'distilbert-base-uncased', output_hidden_states=True)
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased", num_labels=2)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


In [8]:
# load class for hyperparameters
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
    evaluation_strategy="epoch",
)

In [9]:
# define evaluation metrics
import numpy as np
import evaluate

metric = evaluate.load("f1")

In [10]:
# copy pasta 🍝 (no idea what this does)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
from transformers import Trainer
# create the trainer to finetune with
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_test,
    eval_dataset=tokenized_train,
    compute_metrics=compute_metrics,
)

In [12]:
# start fine tuning! 🍿
trainer.train()

/home/cas/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 118517
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 44445
  Number of trainable parameters = 66985530
  0%|          | 0/44445 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 3)